In [1]:
from Bio import SeqIO

def calculate_amino_acid_frequencies(fasta_file):
    """
    Calculate the frequencies of each amino acid in a given FASTA file containing multiple amino acid sequences.

    Parameters:
        fasta_file (str): The path to the FASTA file containing the amino acid sequences.

    Returns:
        dict: A dictionary containing the frequencies of each amino acid. The keys are the amino acid letters,
              and the values are the corresponding frequencies.
    """
    amino_acid_freq = {}

    # Initialize the amino acid frequencies to zero
    for aa in "ACDEFGHIKLMNPQRSTVWY":
        amino_acid_freq[aa] = 0

    # Read the FASTA file and count the occurrences of each amino acid
    with open(fasta_file, "r") as file:
        for record in SeqIO.parse(file, "fasta"):
            sequence = str(record.seq)
            for aa in sequence:
                if aa in amino_acid_freq:
                    amino_acid_freq[aa] += 1

    # Calculate the frequencies
    total_aa_count = sum(amino_acid_freq.values())
    for aa, count in amino_acid_freq.items():
        amino_acid_freq[aa] = count / total_aa_count

    return amino_acid_freq



In [3]:
fasta_file_path = "/home/jack/projects/Proteoform-Exploration/data/sORFs_genomic_hg38.prot.20210329.fa"
amino_acid_frequencies_uORFs = calculate_amino_acid_frequencies(fasta_file_path)
print(amino_acid_frequencies_uORFs)

{'A': 0.0928391881219288, 'C': 0.03046501733169559, 'D': 0.026753979278949225, 'E': 0.0469540137842484, 'F': 0.029781097557203533, 'G': 0.09224324998676839, 'H': 0.024133088726046484, 'I': 0.026380744668347025, 'K': 0.036514442392303254, 'L': 0.09759775756796754, 'M': 0.028593345426226366, 'N': 0.01972300905454793, 'P': 0.09045612293784441, 'Q': 0.037456808232332016, 'R': 0.09986259742420003, 'S': 0.09023273205672891, 'T': 0.047088735669474975, 'V': 0.04984572282071885, 'W': 0.02094512901338902, 'Y': 0.012133217949079251}


In [4]:
fasta_file_path = "/home/jack/projects/Proteoform-Exploration/data/gencode.v44.pc_translations.fa"
amino_acid_frequencies_allCDS = calculate_amino_acid_frequencies(fasta_file_path)
print(amino_acid_frequencies_allCDS)

{'A': 0.06876006192267214, 'C': 0.02163705847754712, 'D': 0.048660771023208725, 'E': 0.07268770452090527, 'F': 0.036089872905738896, 'G': 0.06415776321739619, 'H': 0.0258526474124958, 'I': 0.04414223025748746, 'K': 0.058921573330599, 'L': 0.0989582094716952, 'M': 0.022268380536601014, 'N': 0.03668893315257018, 'P': 0.06137660857191122, 'Q': 0.048319987158934784, 'R': 0.056304046777126804, 'S': 0.08308037329122095, 'T': 0.05336578915598416, 'V': 0.059947562618949804, 'W': 0.012176026267799414, 'Y': 0.026604399929155878}


In [6]:
for key in amino_acid_frequencies_uORFs:
    diff = amino_acid_frequencies_uORFs[key] - amino_acid_frequencies_allCDS[key]
    print(key, '\t', diff)

A 	 0.02407912619925666
C 	 0.008827958854148471
D 	 -0.0219067917442595
E 	 -0.02573369073665687
F 	 -0.006308775348535363
G 	 0.028085486769372198
H 	 -0.001719558686449317
I 	 -0.017761485589140435
K 	 -0.022407130938295744
L 	 -0.0013604519037276608
M 	 0.0063249648896253514
N 	 -0.01696592409802225
P 	 0.029079514365933194
Q 	 -0.010863178926602768
R 	 0.04355855064707323
S 	 0.00715235876550796
T 	 -0.006277053486509185
V 	 -0.010101839798230955
W 	 0.008769102745589605
Y 	 -0.014471181980076627


In [7]:
import pandas as pd

In [21]:
with open('/home/jack/projects/Proteoform-Exploration/data/top50_uORFs.txt', 'r') as f:
    uORFs = f.readlines()
uORFs = [uORF.strip() for uORF in uORFs][1:]


In [22]:
def subset_fasta_by_names(names_list, fasta_path, output_path):
    """
    Subsets a FASTA file by keeping only the sequences whose description contains any of the terms in the given list of names.

    Parameters:
        names_list (list): A list of names (strings) to be used for filtering the FASTA file.
        fasta_path (str): Path to the input FASTA file.
        output_path (str): Path to the output file where the subsetted sequences will be saved.

    Returns:
        None. The function writes the subsetted sequences to the output file.
    """

    # Store the names as a set for faster lookups
    names_set = set(names_list)

    # Variables to store the current sequence ID and sequence lines
    current_sequence_id = None
    current_sequence_lines = []

    # Open the input and output files
    with open(fasta_path, 'r') as fasta_file, open(output_path, 'w') as output_file:
        for line in fasta_file:
            line = line.strip()
            if line.startswith('>'):  # FASTA header line
                # If there was a previous sequence, check if it should be written
                if current_sequence_id is not None and any(name in current_sequence_id for name in names_set):
                    output_file.write(current_sequence_id + '\n')
                    output_file.write('\n'.join(current_sequence_lines) + '\n')

                # Reset variables for the new sequence
                current_sequence_id = line
                current_sequence_lines = []
            else:
                # Append sequence lines
                current_sequence_lines.append(line)

        # Write the last sequence (if applicable)
        if current_sequence_id is not None and any(name in current_sequence_id for name in names_set):
            output_file.write(current_sequence_id + '\n')
            output_file.write('\n'.join(current_sequence_lines) + '\n')


In [23]:
subset_fasta_by_names(uORFs, "/home/jack/projects/Proteoform-Exploration/data/sORFs_genomic_hg38.prot.20210329.fa", "/home/jack/projects/Proteoform-Exploration/data/top_uORFs.fa")

In [24]:
fasta_file_path = "/home/jack/projects/Proteoform-Exploration/data/top_uORFs.fa"
amino_acid_frequencies_uORFs = calculate_amino_acid_frequencies(fasta_file_path)
print(amino_acid_frequencies_uORFs)

{'A': 0.0946604405644097, 'C': 0.03568165668460055, 'D': 0.02705309105674551, 'E': 0.04923358034717287, 'F': 0.03146888640747132, 'G': 0.08313876763780327, 'H': 0.023906202415998376, 'I': 0.02344939600040605, 'K': 0.03679829458938179, 'L': 0.0871485128413359, 'M': 0.03314384326464318, 'N': 0.017510912597705815, 'P': 0.08085473555984164, 'Q': 0.03923459547254086, 'R': 0.09491422190640544, 'S': 0.09674144756877474, 'T': 0.05324332555070551, 'V': 0.05111156227794133, 'W': 0.02547964673637194, 'Y': 0.015226880519744189}


In [25]:
for key in amino_acid_frequencies_uORFs:
    diff = amino_acid_frequencies_uORFs[key] - amino_acid_frequencies_allCDS[key]
    print(key, '\t', diff)

A 	 0.025900378641737554
C 	 0.014044598207053428
D 	 -0.021607679966463216
E 	 -0.023454124173732396
F 	 -0.004620986498267574
G 	 0.01898100442040708
H 	 -0.0019464449964974248
I 	 -0.02069283425708141
K 	 -0.02212327874121721
L 	 -0.011809696630359301
M 	 0.010875462728042167
N 	 -0.019178020554864363
P 	 0.019478126987930423
Q 	 -0.009085391686393926
R 	 0.03861017512927864
S 	 0.013661074277553795
T 	 -0.00012246360527864986
V 	 -0.008836000341008475
W 	 0.013303620468572527
Y 	 -0.01137751940941169


In [29]:
from Bio import SeqIO
from scipy.stats import fisher_exact, chi2_contingency

In [30]:
def count_amino_acid(sequence, amino_acid):
    return sequence.count(amino_acid)

def calculate_enrichment(candidate_sequences, annotated_sequences, amino_acid_of_interest, test="fisher"):
    candidate_count = sum(count_amino_acid(seq, amino_acid_of_interest) for seq in candidate_sequences)
    annotated_count = sum(count_amino_acid(seq, amino_acid_of_interest) for seq in annotated_sequences)

    candidate_total = sum(len(seq) for seq in candidate_sequences)
    annotated_total = sum(len(seq) for seq in annotated_sequences)

    # Create a 2x2 contingency table for Fisher's exact test
    contingency_table = [[candidate_count, candidate_total - candidate_count],
                         [annotated_count, annotated_total - annotated_count]]

    if test == "fisher":
        # Perform Fisher's exact test
        odds_ratio, p_value = fisher_exact(contingency_table)
    elif test == "chi2":
        # Perform chi-squared test
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
        odds_ratio = chi2

    return odds_ratio, p_value

def read_fasta_file(filename):
    sequences = []
    with open(filename, "r") as handle:
        for record in SeqIO.parse(handle, "fasta"):
            sequences.append(str(record.seq))
    return sequences

In [35]:

candidate_fasta_file = "/home/jack/projects/Proteoform-Exploration/data/top_uORFs.fa"
annotated_fasta_file = "/home/jack/projects/Proteoform-Exploration/data/gencode.v44.pc_translations.fa"

# Replace this with the amino acid you want to test for enrichment
amino_acid_of_interest = "K"

# Read the fasta files
candidate_sequences = read_fasta_file(candidate_fasta_file)
annotated_sequences = read_fasta_file(annotated_fasta_file)

# Calculate enrichment
odds_ratio, p_value = calculate_enrichment(candidate_sequences, annotated_sequences, amino_acid_of_interest, test="chi2")

print(f"Enrichment of {amino_acid_of_interest}:")
print(f"Odds Ratio: {odds_ratio}")
print(f"P-value: {p_value}")


Enrichment of K:
Odds Ratio: 188.71569766624043
P-value: 6.062773976113845e-43
